# Ideal Transformer Model test

In [ ]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/dpsim/examples/cxx

SP_ITM_withPiLine_Ph1

In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import numpy as np
import dpsimpy

# %matplotlib widget

epsilon = 1e-12

f_1 = 50
w_s1 = 2 * np.pi * f_1

vs_ref_rms = 100000 * dpsimpy.PEAK1PH_TO_RMS3PH
R_1_R = 6000

# Parameters per Km from KRK Two area system
r = 0.0529
x_l = 0.529
b = 3.308e-6

l_length = 220

R_l = l_length * r
L_l = l_length * (x_l / w_s1)
C_l = l_length * (b / w_s1)
G_l = 0

print(f"Line parameters for {l_length} km:")
print(f"Resistance: {R_l} Ohm")
print(f"Inductance: {L_l} H")
print(f"Capacitance: {C_l} F")
print(f"Conductance: {G_l} S")

resistance = dpsimpy.Math.single_phase_parameter_to_three_phase(R_l)
inductance = dpsimpy.Math.single_phase_parameter_to_three_phase(L_l)
capacitance = dpsimpy.Math.single_phase_parameter_to_three_phase(C_l)
conductance = dpsimpy.Math.single_phase_parameter_to_three_phase(G_l)

R_load = 10000

time_step = 0.00005
final_time = 0.1

In [ ]:
sim_name = "EMT_ITM_Ph3_n1"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

n0 = dpsimpy.emt.SimNode("n0", dpsimpy.PhaseType.ABC)
n1_1 = dpsimpy.emt.SimNode("n1_1", dpsimpy.PhaseType.ABC)
n1_2 = dpsimpy.emt.SimNode("n1_2", dpsimpy.PhaseType.ABC)
n2 = dpsimpy.emt.SimNode("n2", dpsimpy.PhaseType.ABC)
gnd = dpsimpy.emt.SimNode.gnd

vs = dpsimpy.emt.ph3.VoltageSource("v_1", dpsimpy.LogLevel.debug)
vs.set_parameters(
    dpsimpy.Math.single_phase_variable_to_three_phase(complex(vs_ref_rms, 0)), f_1
)

line = dpsimpy.emt.ph3.PiLine("Line", dpsimpy.LogLevel.debug)
line.set_parameters(resistance, inductance, capacitance, conductance)

itm = dpsimpy.signal.DecouplingIdealTransformer_EMT_Ph3("itm")
itm.set_parameters(
    n1_1, n1_2, 0, np.zeros((3, 1)), np.zeros((3, 1)), dpsimpy.CouplingMethod.DELAY
)

R_1 = dpsimpy.emt.ph3.Resistor("R1")
R_1.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(R_1_R))

load = dpsimpy.emt.ph3.Resistor("R_load", dpsimpy.LogLevel.debug)
load.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(R_load))

vs.connect([gnd, n0])
R_1.connect([n0, n1_1])
line.connect([n1_2, n2])
load.connect([n2, gnd])

sys = dpsimpy.SystemTopology(f_1, [n0, n1_1, n1_2, n2], [vs, R_1, line, itm, load])
sys.add(itm.get_components())

logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1_1", "v", n1_1)
logger.log_attribute("v1_2", "v", n1_2)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("iline", "i_intf", line)
logger.log_attribute("iref_itm", "i_ref", itm)
logger.log_attribute("i_intf_itm", "i_intf", itm)
logger.log_attribute("vref_itm", "v_ref", itm)
logger.log_attribute("v_intf_itm", "v_intf", itm)
logger.log_attribute("iload", "i_intf", load)

sim = dpsimpy.Simulation(sim_name)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.EMT)
sim.add_logger(logger)

sim.run()

In [ ]:
sys

In [ ]:
sim_name = "SP_ITM_Ph1_n1"
dpsimpy.Logger.set_log_dir("logs/" + sim_name)

n0 = dpsimpy.sp.SimNode("n0")
n1_1 = dpsimpy.sp.SimNode("n1_1")
n1_2 = dpsimpy.sp.SimNode("n1_2")
n2 = dpsimpy.sp.SimNode("n2")
gnd = dpsimpy.sp.SimNode.gnd

vs = dpsimpy.sp.ph1.VoltageSource("v_1", dpsimpy.LogLevel.debug)
vs.set_parameters(complex(dpsimpy.RMS3PH_TO_PEAK1PH * vs_ref_rms, 0), f_1)

line = dpsimpy.sp.ph1.PiLine("Line", dpsimpy.LogLevel.debug)
line.set_parameters(R_l, L_l, C_l, G_l)

itm = dpsimpy.signal.DecouplingIdealTransformer_SP_Ph1("itm", dpsimpy.LogLevel.debug)
itm.set_parameters(
    n1_1, n1_2, 0, np.zeros((1, 1)), complex(0, 0), dpsimpy.CouplingMethod.DELAY
)

R_1 = dpsimpy.sp.ph1.Resistor("R1")
R_1.set_parameters(R_1_R)

load = dpsimpy.sp.ph1.Resistor("R_load", dpsimpy.LogLevel.debug)
load.set_parameters(R_load)

vs.connect([gnd, n0])
R_1.connect([n0, n1_1])
line.connect([n1_2, n2])
load.connect([n2, gnd])

sys = dpsimpy.SystemTopology(f_1, [n0, n1_1, n1_2, n2], [vs, R_1, line, itm, load])
sys.add(itm.get_components())
sys.add_node(itm.get_virtual_node())

logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1_1", "v", n1_1)
logger.log_attribute("v1_2", "v", n1_2)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("iline", "i_intf", line)
logger.log_attribute("iref_itm", "i_ref", itm)
logger.log_attribute("i_intf_itm", "i_intf", itm)
logger.log_attribute("vref_itm", "v_ref", itm)
logger.log_attribute("v_intf_itm", "v_intf", itm)
logger.log_attribute("iload", "i_intf", load)

sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.debug)
sim.set_system(sys)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.SP)
sim.do_split_subnets(True)
sim.add_logger(logger)

sim.run()

In [ ]:
sys

In [ ]:
work_dir = "logs/EMT_ITM_Ph3_n1/"
log_name = "EMT_ITM_Ph3_n1"
print(work_dir + log_name + ".csv")
emt_itm_ph3_piline = rt.read_timeseries_dpsim(work_dir + log_name + ".csv")

In [ ]:
work_dir = "logs/SP_ITM_Ph1_n1/"
log_name = "SP_ITM_Ph1_n1"
print(work_dir + log_name + ".csv")
sp_itm_ph1_piline = rt.read_timeseries_dpsim(work_dir + log_name + ".csv")

In [ ]:
work_dir = "logs/SP_DecouplingITM_Ph1_withPiLine_Monolithic/"
log_name = "SP_DecouplingITM_Ph1_withPiLine_Monolithic"
print(work_dir + log_name + ".csv")
sp_itm_ph1_piline_cxx_monolithic = rt.read_timeseries_dpsim(
    work_dir + log_name + ".csv"
)

In [ ]:
plt.figure()
plt.plot(
    sp_itm_ph1_piline_cxx_monolithic["v1"].time,
    sp_itm_ph1_piline_cxx_monolithic["v1"].values,
)
plt.plot(emt_itm_ph3_piline["v1_1_0"].time, emt_itm_ph3_piline["v1_1_0"].values)
plt.plot(emt_itm_ph3_piline["v1_2_0"].time, emt_itm_ph3_piline["v1_2_0"].values, "--")
plt.plot(sp_itm_ph1_piline["v1_1"].time, sp_itm_ph1_piline["v1_1"].values, "--")
plt.xlabel("time [s]")
plt.ylabel("Voltage [V]")
plt.title("V_1")
plt.legend(
    ["Monolithic", "v1_1 ITM - EMT - Ph3", "v1_2 ITM - EMT - Ph3", "ITM  SP - Ph1"]
)
plt.show()

plt.figure()
plt.plot(
    sp_itm_ph1_piline_cxx_monolithic["v2"].time,
    sp_itm_ph1_piline_cxx_monolithic["v2"].values,
)
plt.plot(emt_itm_ph3_piline["v2_0"].time, emt_itm_ph3_piline["v2_0"].values, "--")
plt.plot(sp_itm_ph1_piline["v2"].time, sp_itm_ph1_piline["v2"].values, "-.")
plt.xlabel("time [s]")
plt.ylabel("Voltage [V]")
plt.title("V_2")
plt.legend(["Monolithic", "ITM - EMT - Ph3", "ITM  SP - Ph1"])
plt.show()
# plt.savefig('piline_split_itm.svg')

In [ ]:
plt.figure()
plt.plot(
    emt_itm_ph3_piline["i_intf_itm_0"].time, emt_itm_ph3_piline["i_intf_itm_0"].values
)
plt.plot(
    sp_itm_ph1_piline["i_intf_itm"].time, sp_itm_ph1_piline["i_intf_itm"].values, "-."
)
plt.xlabel("time [s]")
plt.ylabel("Current [A]")
plt.title("I_intf_ITM")

plt.figure()
plt.plot(
    emt_itm_ph3_piline["v_intf_itm_0"].time, emt_itm_ph3_piline["v_intf_itm_0"].values
)
plt.plot(
    sp_itm_ph1_piline["v_intf_itm"].time, sp_itm_ph1_piline["v_intf_itm"].values, "-."
)
plt.xlabel("time [s]")
plt.ylabel("Voltage [V]")
plt.title("V_intf_ITM")